# Graph Classification Problem

Uses Proteins Dataset given in the Assignment pdf. 

In [ ]:
!pip install spektral


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 140 kB 5.6 MB/s 


In [ ]:
from spektral.datasets import TUDataset

data = TUDataset('PROTEINS')
data


100%|█████████████████████████████████████████| 447k/447k [00:00<00:00, 800kB/s]


Successfully loaded PROTEINS.


TUDataset(n_graphs=1113)

In [ ]:
from spektral.transforms import GCNFilter

data.apply(GCNFilter())


In [ ]:
import numpy as np

np.random.shuffle(data)
split = int(0.8 * len(data))
data_train, data_test = data[:split], data[split:]

<ipython-input-4-4288fec95ac6>:3: UserWarning: you are shuffling a 'TUDataset' object which is not a subclass of 'Sequence'; `shuffle` is not guaranteed to behave correctly. E.g., non-numpy array/tensor objects with view semantics may contain duplicates after shuffling.
  np.random.shuffle(data)


In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout
from spektral.layers import GCNConv, GlobalSumPool

**Model 01**

In [ ]:
class ProteinsGNN(Model):
  
  def __init__(self, n_hidden, n_labels):
    super().__init__()
    self.graph_conv = GCNConv(n_hidden)
    self.pool = GlobalSumPool()
    self.dropout = Dropout(0.5)
    self.dense = Dense(n_labels, 'softmax')

  def call(self, inputs):
    out = self.graph_conv(inputs)
    out = self.dropout(out)
    out = self.pool(out)
    out = self.dense(out)

    return out

In [ ]:
model1 = ProteinsGNN(32, data.n_labels)

In [ ]:
model1.compile('adam', 'categorical_crossentropy')

In [ ]:
from spektral.data import BatchLoader

loader = BatchLoader(data_train, batch_size=32)

In [ ]:
model1.fit(loader.load(), steps_per_epoch=loader.steps_per_epoch, epochs=30)

Epoch 1/30
28/28 [==============================] - 1s 28ms/step - loss: 0.7895
Epoch 2/30
28/28 [==============================] - 1s 23ms/step - loss: 0.6990
Epoch 3/30
28/28 [==============================] - 1s 24ms/step - loss: 0.7030
Epoch 4/30
28/28 [==============================] - 1s 22ms/step - loss: 0.7532
Epoch 5/30
28/28 [==============================] - 1s 32ms/step - loss: 0.7580
Epoch 6/30
28/28 [==============================] - 1s 20ms/step - loss: 0.6956
Epoch 7/30
28/28 [==============================] - 1s 23ms/step - loss: 0.7148
Epoch 8/30
28/28 [==============================] - 1s 25ms/step - loss: 0.9733
Epoch 9/30
28/28 [==============================] - 1s 24ms/step - loss: 0.7545
Epoch 10/30
28/28 [==============================] - 1s 25ms/step - loss: 0.6483
Epoch 11/30
28/28 [==============================] - 1s 22ms/step - loss: 0.6979
Epoch 12/30
28/28 [==============================] - 1s 23ms/step - loss: 0.6715
Epoch 13/30
28/28 [==================

In [ ]:
test_loader = BatchLoader(data_test, batch_size=32)

In [ ]:
loss = model1.evaluate(test_loader.load(), steps=loader.steps_per_epoch)

print('Test loss: {}'.format(loss))

28/28 [==============================] - 0s 16ms/step - loss: 0.6894
Test loss: 0.6893763542175293


**Model** **02** 

In [ ]:
class ProteinsGNNModel2(Model):
  
  def __init__(self, n_hidden, n_labels):
    super().__init__()
    self.graph_conv = GCNConv(n_hidden)
    self.pool = GlobalSumPool()
    self.dropout = Dropout(0)
    self.dense = Dense(n_labels, 'softmax')

  def call(self, inputs):
    out = self.graph_conv(inputs)
    out = self.dropout(out)
    out = self.pool(out)
    out = self.dense(out)

    return out

In [ ]:
model = ProteinsGNNModel2(16, data.n_labels)
model.compile('adam', 'categorical_crossentropy')
loader = BatchLoader(data_train, batch_size=32)
model.fit(loader.load(), steps_per_epoch=loader.steps_per_epoch, epochs=10)
test_loader = BatchLoader(data_test, batch_size=32)
loss = model.evaluate(loader.load(), steps=loader.steps_per_epoch)
print('Test loss: {}'.format(loss))

Epoch 1/10
28/28 [==============================] - 1s 25ms/step - loss: 80.8237
Epoch 2/10
28/28 [==============================] - 1s 21ms/step - loss: 16.4794
Epoch 3/10
28/28 [==============================] - 1s 23ms/step - loss: 6.0820
Epoch 4/10
28/28 [==============================] - 1s 24ms/step - loss: 1.2891
Epoch 5/10
28/28 [==============================] - 1s 21ms/step - loss: 0.8883
Epoch 6/10
28/28 [==============================] - 1s 22ms/step - loss: 0.8615
Epoch 7/10
28/28 [==============================] - 1s 19ms/step - loss: 0.6555
Epoch 8/10
28/28 [==============================] - 1s 20ms/step - loss: 0.6714
Epoch 9/10
28/28 [==============================] - 1s 18ms/step - loss: 0.6902
Epoch 10/10
28/28 [==============================] - 1s 17ms/step - loss: 0.6811
Test loss: 0.6811217665672302


In [ ]:
#Getting labels of the graphs
#Takes a long time due to the size of the dataset 
#Still identifies as a unknown graph 
output = model1.predict(test_loader)
output


     15/Unknown - 0s 13ms/step

/usr/local/lib/python3.8/dist-packages/spektral/data/utils.py:221: UserWarning: you are shuffling a 'TUDataset' object which is not a subclass of 'Sequence'; `shuffle` is not guaranteed to behave correctly. E.g., non-numpy array/tensor objects with view semantics may contain duplicates after shuffling.
  np.random.shuffle(a)


  85460/Unknown - 1248s 15ms/step

In [ ]:
output = model1.predict(test_loader)
output